In [3]:
# Import Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [5]:


# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_set = train_datagen.flow_from_directory(
    "C:/Users/HP/Downloads/archive/dataset/train",  # Replace with your training data path
    target_size=(224, 224),  # Resize images to match MobileNet input
    batch_size=32,
    class_mode='binary'  # Binary classification
)

Found 11170 images belonging to 2 classes.


In [9]:
test_set = test_datagen.flow_from_directory(
    "C:/Users/HP/Downloads/archive/dataset/test",  # Replace with your test data path
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 2792 images belonging to 2 classes.


In [11]:
# Display Class Indices
print("Class Indices:", train_set.class_indices)

Class Indices: {'millitary': 0, 'other': 1}


In [13]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 11s 1us/step


In [15]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Add a global average pooling layer
    Dense(512, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [17]:
# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional)      │ (None, 7, 7, 1024)          │       3,228,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,754,177 (14.32 MB)

 Trainable params: 525,313 (2.00 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [21]:
history = model.fit(
    train_set,
    validation_data=test_set,
    epochs=10,  # Train for 10 epochs (adjust as needed)
    steps_per_epoch=train_set.samples // train_set.batch_size,
    validation_steps=test_set.samples // test_set.batch_size
)

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
349/349 ━━━━━━━━━━━━━━━━━━━━ 469s 1s/step - accuracy: 0.7527 - loss: 0.4892 - val_accuracy: 0.9253 - val_loss: 0.1944
Epoch 2/10
  1/349 ━━━━━━━━━━━━━━━━━━━━ 2:51 493ms/step - accuracy: 0.9375 - loss: 0.1871

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


349/349 ━━━━━━━━━━━━━━━━━━━━ 43s 122ms/step - accuracy: 0.9375 - loss: 0.1871 - val_accuracy: 0.9256 - val_loss: 0.1939
Epoch 3/10
349/349 ━━━━━━━━━━━━━━━━━━━━ 376s 1s/step - accuracy: 0.9010 - loss: 0.2366 - val_accuracy: 0.9303 - val_loss: 0.1715
Epoch 4/10
349/349 ━━━━━━━━━━━━━━━━━━━━ 44s 126ms/step - accuracy: 0.9062 - loss: 0.1853 - val_accuracy: 0.9300 - val_loss: 0.1719
Epoch 5/10
349/349 ━━━━━━━━━━━━━━━━━━━━ 1019s 3s/step - accuracy: 0.9146 - loss: 0.2059 - val_accuracy: 0.9407 - val_loss: 0.1533
Epoch 6/10
349/349 ━━━━━━━━━━━━━━━━━━━━ 45s 128ms/step - accuracy: 0.9375 - loss: 0.1610 - val_accuracy: 0.9400 - val_loss: 0.1526
Epoch 7/10
349/349 ━━━━━━━━━━━━━━━━━━━━ 371s 1s/step - accuracy: 0.9234 - loss: 0.1936 - val_accuracy: 0.9382 - val_loss: 0.1510
Epoch 8/10
349/349 ━━━━━━━━━━━━━━━━━━━━ 45s 128ms/step - accuracy: 0.8750 - loss: 0.2256 - val_accuracy: 0.9389 - val_loss: 0.1508
Epoch 9/10
349/349 ━━━━━━━━━━━━━━━━━━━━ 371s 1s/step - accuracy: 0.9265 - loss: 0.1859 - val_accura

In [27]:
model.evaluate(test_set)


88/88 ━━━━━━━━━━━━━━━━━━━━ 44s 503ms/step - accuracy: 0.9405 - loss: 0.1441


[0.14270152151584625, 0.9437679052352905]

In [31]:
# Evaluate the Model
loss, accuracy = model.evaluate(test_set)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

88/88 ━━━━━━━━━━━━━━━━━━━━ 45s 506ms/step - accuracy: 0.9410 - loss: 0.1525
Test Accuracy: 94.38%


In [32]:

model.save("military_vs_transport_model.h5")

In [35]:
def predict_image(image_path, model):
    # Load and preprocess the image
    test_image = load_img(image_path, target_size=(224, 224))  # Resize to match model input
    test_image = img_to_array(test_image) / 255.0  # Normalize pixel values
    test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension

    # Make prediction
    result = model.predict(test_image)
    confidence = result[0][0]
    if confidence < 0.5:
        prediction = "military"
    else:
        prediction = "other transport"
    return prediction, confidence

In [45]:
image_path = "C:/Users/HP/Downloads/military.webp"  # Replace with your image path
prediction, confidence = predict_image(image_path, model)
print(f"Prediction: {prediction}, Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Prediction: military, Confidence: 0.00


In [47]:
image_path = "C:/Users/HP/Downloads/v.webp"  # Replace with your image path
prediction, confidence = predict_image(image_path, model)
print(f"Prediction: {prediction}, Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Prediction: other transport, Confidence: 1.00


In [2]:
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from PIL import Image

# Load the trained model
model = load_model("military_vs_transport_model.h5")

# Set the title and description
st.title("🚀 Military vs. Transport Vehicle Classifier")
st.write("Upload an image, and the AI model will classify it as either **Military** or **Other Transport**.")

# File uploader for user image input
uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

# Prediction function
def predict_image(image):
    image = image.resize((224, 224))  # Resize to match model input
    image = img_to_array(image) / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    
    # Make prediction
    result = model.predict(image)
    confidence = result[0][0]
    
    if confidence < 0.5:
        return "🚔 Military Vehicle", confidence
    else:
        return "🚗 Other Transport", confidence

# Display uploaded image and predictions
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)
    
    # Run prediction
    prediction, confidence = predict_image(image)
    
    # Show results
    st.markdown(f"### **Prediction: {prediction}**")
    st.markdown(f"### **Confidence: {confidence:.2f}**")


2025-02-09 21:40:11.218 
  command:

    streamlit run C:\Users\HP\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
!jupyter nbconvert --to script dl-Copy1.ipynb
